In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [3]:
%%capture pwd
!pwd

In [4]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [18]:
df = (
    pd.read_csv("raw/ADS_bigcatexp.csv")
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
# df.columns = ["response", "percent"]
# df["percent"] *= 100
df = df.set_index("Unnamed: 0").stack().reset_index()
df.columns = ["parent", "id", "value"]

In [36]:
c = {
    "High sugar and alcohol": colors["eco-dot"],
    "Processed": colors["eco-background"],
    "Fast food": colors["eco-yellow"],
    "Home cooking": colors["eco-green"],
    "Supermarkets": colors["eco-turquiose"],
}

In [37]:
df = df.join(pd.DataFrame(c, index=["color"]).T, on="parent")

In [38]:
f = "fig1_food groups"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".json", "w").write(
    json.dumps(list(df.T.to_dict().values()))
)
if LOCAL:
    f1 = df
df.head()

,parent,id,value,color
0,High sugar and alcohol,Confectionery,153.1770,#F4134D
1,High sugar and alcohol,Soft Drinks,64.5109,#F4134D
2,High sugar and alcohol,Alcohol,54.2943,#F4134D
3,High sugar and alcohol,Desserts,32.0924,#F4134D
4,High sugar and alcohol,Biscuits,8.8821,#F4134D
